In [2]:
24 * 60 * 60 / 8

10800.0

In [1]:
import json
import random
from pathlib import Path
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
from tqdm.notebook import tqdm

# ===================== Конфигурация =====================
DATA_DIR = Path("/var/lib/kinsovet/data/movies")  # путь к JSON файлам
MODEL_NAME = "Qwen/Qwen3-0.6B"

# ===================== Загрузка модели =====================
print(f"Loading {MODEL_NAME}...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto"
)

# ===================== Функция рерайта =====================
def rewrite_description(text: str) -> str:
    """Перепиши описание фильма для коммерческого использования."""
    prompt = f"""Ты — профессиональный копирайтер. Перепиши описание фильма своими словами для коммерческого использования на сайте рекомендаций.

Требования:
- Пиши на русском языке
- Делай текст уникальным, меняй формулировки
- Сохраняй сюжет и ключевые детали
- Избегай спойлеров
- Делай текст живым и интересным
- Итоговая длина не более 4-х предложений, можно короче.

Исходный текст:
{text}
"""

    messages = [
        {"role": "user", "content": prompt}
    ]
    
    text_input = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True
    )
    model_inputs = tokenizer([text_input], return_tensors="pt").to(model.device)
    
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
    
    # Parsing thinking content
    try:
        index = len(output_ids) - output_ids[::-1].index(151668)
    except ValueError:
        index = 0
    
    # thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
    
    return content

# ===================== Обработка файлов =====================
json_files = list(DATA_DIR.glob("*.json"))

if len(json_files) == 0:
    print("❌ No JSON files found! Check DATA_DIR path.")
else:
    selected_files = random.sample(json_files, min(10, len(json_files)))
    print(f"Processing {len(selected_files)} files...")
    
    results = []
    
    for i, file_path in tqdm(enumerate(selected_files, 1)):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            original = data.get("description", {}).get("original", "")
            
            if original:
                print(f"[{i}/{len(selected_files)}] Processing {file_path.name}...")
                rewritten = rewrite_description(original)
                results.append({
                    "File": file_path.name,
                    "Исходный": original,
                    "Рерайт": rewritten
                })
                print(f"    ✓ Done")
            else:
                print(f"    ⚠ No 'description.original' found")
        except Exception as e:
            print(f"    ❌ Error: {e}")
    
    # ===================== Вывод таблицы =====================
    if results:
        print("\n" + "="*80)
        print("РЕЗУЛЬТАТЫ")
        print("="*80)
        
        df = pd.DataFrame(results)
        pd.set_option('display.max_colwidth', None)
        pd.set_option('display.width', None)
        display(df[["Исходный", "Рерайт"]])
        
        # Опционально: сохранить в CSV
        # df.to_csv("rewrite_results.csv", index=False, encoding="utf-8")
        # print("\nSaved to rewrite_results.csv")
    else:
        print("❌ No valid descriptions found.")

Loading Qwen/Qwen3-0.6B...


Loading weights:   0%|          | 0/311 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


Processing 10 files...


0it [00:00, ?it/s]

[1/10] Processing 5080565.json...
    ✓ Done
[2/10] Processing 972247.json...
    ✓ Done
[3/10] Processing 991738.json...
    ✓ Done
    ⚠ No 'description.original' found
[5/10] Processing 1228689.json...
    ✓ Done
    ⚠ No 'description.original' found
[7/10] Processing 934350.json...
    ✓ Done
[8/10] Processing 225436.json...
    ✓ Done
[9/10] Processing 50686.json...
    ✓ Done
[10/10] Processing 911530.json...
    ✓ Done

РЕЗУЛЬТАТЫ


,Исходный,Рерайт
0,"Юная Дорис, чей брат тяжело болен, завязывает дружбу с эксцентричным активистом.","Юная Дорис и её брат, который болен, обсуждают дружбу с экспериментальным активистом. Это не просто взаимодействие — оно доказывает"
1,"Бессмертие. Не этого ли всегда жаждало человечество? Да, так было ровно до тех пор, пока по всему миру не начали появляться люди, неспособные умереть. Солдаты богов, полулюди. Загадка тысячелетия. Необъяснимый феномен эволюции и любимая игрушка учёных. Самое пугающее свойство полулюдей в том, что ими может оказаться кто угодно. «Ты не узнаешь правду, пока не умрёшь». Нагаи Кей умер... и он больше никогда не умрёт.","Время, когда человечество ждала сверхъестественных с"
2,"За сто лет жизни простой украинский крестьянин Яков испытал все, о чем только может мечтать человек. Но на его долю выпало и то, чего не пожелаешь даже врагу. Трогательная и реалистическая любовная драма, которая началась на Западной Украине времен польского владычества и продлилась до наших дней.","Крестьянин Яков, с каждым годом сталкиваясь с реальностью, сталкивается с тем, что его желания не могут быть реализованы. Его драматичная история, начиная с исторических событий на Западной Украине и идущая до наших дней, создает глубокую и живую реалистичную мотивацию."
3,"Однажды дочь случайно читает письмо матери и узнает тайну, которую та скрывала всю жизнь. Девочка хочет утешить мать и предлагает ей отправиться в путешествие.","Дочь, взбешенная от тайны матери, решает отправиться в путешествие, чтобы найти близкое и утешить. Взбешенная она почувствовала себя в тени, а сила желания превратилась в восторжество."
4,"Анна и Франц жили в небольшом немецком городе, были молоды, влюблены друг в друга и собирались пожениться. Но в тихое счастье двоих вмешался большой сумасшедший мир – шла Первая мировая война, Франца отправили на фронт во Францию, где он, 23-летний скрипач, обожавший стихи Верлена, погиб. Война закончилась, а жизнь Анны теперь свелась к поддержке пожилых родителей любимого и посещениям кладбища. Однажды там она видит незнакомого человека, который принес цветы на могилу Франца. Он, Адриен, – француз, а, значит, враг. Но встреча с ним переворачивает ее жизнь.","Анна и Франц жили в небольшом городе, влюблясь и поженившись. Война вступила, но Франц погиб, и Анна теперь учится поддерживать родителей и участвовать в кладбище. Однажды она увидела незнакомого человека, который принес цветы на могилу — Адриен, противника, но его встреча переворачивает судьбу."
5,"Герой фильма на студенческие каникулы приезжает в деревню. Это лето останется в его памяти на всю жизнь: он встречается с огромной любовью, которую теряет из-за того, что не может решить, кто же влечет его больше: страстная, открытая Соня или загадочная мечтательная Натали.","Герой студенческих каникул приезжает в деревню, где лето останется в его памяти — он встречается с любовью, которую он не может остановить. Соня — взмезанный, а Натали — мечтательная, и он пытается найти, кто из них влечет его больше."
6,"1984 год. По политическим мотивам Бедржих Мара уходит с поста главы Пражской Академии искусств. Вместе с женой и двумя сыновьями он ютится в маленькой квартирке. Полная противоположность его семьи — дом Мила Бржечка, директора школы и активиста коммунистического движения.","Все в тени Пражской Академии — в квартире с двумя детьми и женой, противоположность семьи Мила Бржечки, директора и активиста коммунистического движения."
7,"Троица уголовников получает задание от наркобарона – похитить сына его бывшего партнера. Когда дело идет наперекосяк, похитители пускаются в бега, готовые на всё, лишь бы вновь не оказаться за решеткой.","Троица уголовников задана похитить сына бывшего партнера, когда дело идет наперекосяк, и похитители готовы на всё, лишь бы не оказаться за решеткой."
